# Maximal Marginal Relevance technique

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

d:\RAG\RAG-Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loader = TextLoader("langchain.txt")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap = 50,
    separators = ["\n\n", "\n"," ",""]
)

docs = splitter.split_documents(documents)

docs

[Document(metadata={'source': 'langchain.txt'}, page_content='# LangChain Modules and Use Cases\n\nLangChain provides support for several main modules, each designed to address specific aspects of building applications with language models. For each module, LangChain offers examples to get started, how-to guides, reference documentation, and conceptual overviews. Below is a detailed breakdown of these modules, organized in increasing order of complexity.\n\n---\n\n## 1. Prompts'),
 Document(metadata={'source': 'langchain.txt'}, page_content='---\n\n## 1. Prompts\n\nThe **Prompts** module focuses on prompt design, management, and optimization. It includes tools and utilities for:'),
 Document(metadata={'source': 'langchain.txt'}, page_content='- **Prompt Templates**: Construct templates that accept variables, few-shot exemplars, and structured prompts for different LLM APIs.\n- **Utilities**: Support for prompt serialization, versioning, A/B testing, and guarding against prompt injectio

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embedding=embedding_model)


In [6]:
# Create MMR retriever

retriever = vector_store.as_retriever(
    search_type= "mmr",
    search_kwargs={"k":3}
)

In [7]:
prompt = PromptTemplate.from_template(
"""
Anser the question based on the context provided.

Context:
{context}

Question: {input}
"""
)

llm= init_chat_model("groq:openai/gpt-oss-20b")

In [9]:
# RAG Pipeline
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [10]:
response = retrieval_chain.invoke({"input":"How does Langchain support agents and memory?"})

print("Answer :\n", response["answer"])

Answer :
 **LangChain’s support for agents and memory**

| Feature | How LangChain implements it |
|---------|-----------------------------|
| **Agents** | • The **Agents** module implements a *decision‑making loop*: an LLM selects an action, a tool is executed, the tool’s observation is fed back to the LLM, and the cycle repeats until a termination condition is reached. <br>• Agents can use any tool that LangChain exposes (HTTP calls, database queries, custom code, etc.). <br>• The framework ships with pre‑built agent templates (e.g., *Zero‑Shot ReAct*, *Tool‑Calling*, *Planner‑Executor*) and the ability to plug in custom logic. |
| **Memory for Chatbots** | • **Ephemeral in‑memory buffers** for short‑term context. <br>• **Key‑value stores** (e.g., Redis, SQLite) to persist chat state across sessions. <br>• **Vector‑based memories** that embed past turns and allow similarity‑search retrieval. <br>• **Summary‑based long‑term memory** that condenses a long conversation into a concise su

In [11]:
response

{'input': 'How does Langchain support agents and memory?',
 'context': [Document(id='c1963cdd-8812-4b85-812c-5d6cd7f55e61', metadata={'source': 'langchain.txt'}, page_content='---\n\n# Use Cases\n\nLangChain modules can be used in a variety of ways. Below are some of the common use cases supported by LangChain:\n\n### 1. Agents\nAgents use a language model to interact with other tools. They can be used for grounded question/answering, interacting with APIs, or taking actions.\n\n### 2. Chatbots\nLanguage models are ideal for creating chatbots due to their ability to produce coherent and contextually relevant text.'),
  Document(id='baf13151-3855-49a4-bc33-07e5c22ade8a', metadata={'source': 'langchain.txt'}, page_content='- **Implementations**: Ephemeral in-memory buffers, key-value stores, vectorized memories, and summary-based long-term memory.\n- **Capabilities**: Memory retrieval strategies, staleness management, privacy-preserving storage, and configurable retention policies.\n- **